In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = os.getcwd().replace('scripts','')
reviews = pd.read_csv(path + 'data/Movies_Reviews.csv')
reviews_sample = reviews.sample( n = 5000 )


In [ ]:
reviews_sample["averageRating"].hist( density = True )
reviews["averageRating"].hist( density = True)
plt.show()


In [ ]:
features = reviews_sample['Plot']
targets = reviews_sample['averageRating']
features = features.str.lower()
features = features.str.replace('[^\w\s]','')

In [ ]:
import torch
from transformers import *
#from pytorch_pretrained_bert import BertTokenizer 
tokenizer = BertTokenizer.from_pretrained(  'bert-base-uncased'  )
model = BertModel.from_pretrained( "bert-base-uncased" )
input_ids = features.apply( lambda x:  tokenizer.encode(   " ".join( tokenizer.tokenize( x )[:400]   ) ))     
#input_ids = torch.tensor( [   tokenizer.tokenize( x ) for x in features.values[:10]  ] ) 

In [12]:
len( input_ids )

5000

In [ ]:
new_features = []
valid_index = []
i = 0 
model = BertModel.from_pretrained(  'bert-base-uncased' , output_hidden_states = True )
for feat in tqdm( input_ids ) : 
    
    try:
        inputs_tensor = torch.tensor( [ feat ])
        all_hidden_states, all_attentions = model(inputs_tensor)[-2:]
        new_features.append( all_hidden_states )
        valid_index.append( i )
        i = i + 1 
    except:
        continue
    

  0%|          | 13/5000 [00:18<2:10:37,  1.57s/it]

In [ ]:
input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.


In [ ]:
input_ids

In [ ]:
tokenizer.tokenize(" asdasdas")

In [ ]:
import torch
from transformers import *

# Transformers has a unified API
# for 8 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [(BertModel,       BertTokenizer,       'bert-base-uncased')]

# To use TensorFlow 2.0 versions of the models, simply prefix the class names with 'TF', e.g. `TFRobertaModel` is the TF 2.0 counterpart of the PyTorch model `RobertaModel`

# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

    # Each architecture is provided with several class for fine-tuning on down-stream tasks, e.g.
    BERT_MODEL_CLASSES = [BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
                          BertForSequenceClassification, BertForMultipleChoice, BertForTokenClassification,
                          BertForQuestionAnswering]

    BERT_MODEL_CLASSES = [ BertModel ]
    # All the classes for an architecture can be initiated from pretrained weights for this architecture
    # Note that additional weights added for fine-tuning are only initialized
    # and need to be trained on the down-stream task
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    for model_class in BERT_MODEL_CLASSES:
        # Load pretrained model/tokenizer
        #model = model_class.from_pretrained('bert-base-uncased')

        # Models can return full list of hidden-states & attentions weights at each layer
        model = model_class.from_pretrained(pretrained_weights,
                                            output_hidden_states=True,
                                            output_attentions=False)
        input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
        all_hidden_states, all_attentions = model(input_ids)[-2:]

        print( all_hidden_states   )
        print( all_hidden_states.shape )
        # Models are compatible with Torchscript
        #model = model_class.from_pretrained(pretrained_weights, torchscript=True)
        #traced_model = torch.jit.trace(model, (input_ids,))

        # Simple serialization for models and tokenizers
        #model.save_pretrained('./directory/to/save/')  # save
        #model = model_class.from_pretrained('./directory/to/save/')  # re-load
        #tokenizer.save_pretrained('./directory/to/save/')  # save
        #tokenizer = BertTokenizer.from_pretrained('./directory/to/save/')  # re-load

        # SOTA examples for GLUE, SQUAD, text generation...